In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import pyro
import pyro.distributions as dist
import pyro.infer
import pyro.optim
from pyro.optim import Adam
import torch.distributions.constraints as constraints
import scipy
import scipy.stats


def scale(Data):
    Jskill = pyro.sample('JS', dist.Normal(120.,40.))
    Jperf = pyro.sample('JP', dist.Normal(Jskill, 5.))
    Fskill = pyro.sample('FS', dist.Normal(100.,5.)) 
    Fperf = pyro.sample('FP', dist.Normal(Fskill, 5.))
    Jwin = torch.tensor([0.0])
    if Fperf<Jperf:
        Jwin = torch.tensor([1.0])
    with pyro.plate("plate", len(Data)):
        a= pyro.sample('JW', dist.Bernoulli(Jwin), obs=Data)

def guide(Data):
    J_m = pyro.param('J_m', torch.tensor(0.0))
    J_s = pyro.param('J_s', torch.tensor(5.0), constraint = constraints.positive)
    F_m = pyro.param('F_m', torch.tensor(0.0))
    F_s = pyro.param('F_s', torch.tensor(5.0), constraint=constraints.positive)
    Jp_s = pyro.param('JP_s', torch.tensor(25.0), constraint=constraints.positive)
    Fp_s = pyro.param('FP_s', torch.tensor(25.0),constraint=constraints.positive)
    Jskill = pyro.sample('JS', dist.Normal(J_m,J_s))
    Jperf = pyro.sample('JP', dist.Normal(Jskill, Jp_s))
    Fskill = pyro.sample('FS', dist.Normal(F_m,F_s))
    Fperf = pyro.sample('FP', dist.Normal(Fskill,Fp_s))
    
Data =  torch.tensor([1.0])

pyro.clear_param_store()
adam_params = {"lr": 0.09, "betas": (.9, .999)}
optimizer = Adam(adam_params)
svi = pyro.infer.SVI(model=scale,
                     guide=guide,
                     optim=optimizer,
                     loss=pyro.infer.TraceGraph_ELBO())


losses, a,b,c,d  = [], [], [], [], []
num_steps = 5000
for t in range(num_steps):
    losses.append(svi.step(Data))
    a.append(pyro.param("J_s").item())
    b.append(pyro.param("J_m").item())
    c.append(pyro.param("F_s").item())
    d.append(pyro.param("F_m").item())
    if t%500 ==0: 
        print('.')

plt.subplot(5,1,1)
plt.plot(losses)
plt.title("Adam")
plt.xlabel("step")
plt.ylabel("loss");

plt.subplot(5,1,2)
plt.plot(a)
plt.xlabel('step')
plt.ylabel('Js')

plt.subplot(5,1,3)
plt.plot(b)
plt.xlabel('step')
plt.ylabel('Jm')

plt.subplot(5,1,4)
plt.plot(c)
plt.xlabel('step')
plt.ylabel('Fs')

plt.subplot(5,1,5)
plt.plot(d)
plt.xlabel('step')
plt.ylabel('Fm')

plt.savefig('N1loss.png')
print('Jskills = N({1},{0} ), Fskills=N({3},{2}) '.format(pyro.param("J_s").item(), pyro.param("J_m").item(), pyro.param("F_s").item(), pyro.param("F_m").item()))


print("the book obtains the solution N(140.1, 28.5)")
print((140.1- pyro.param("J_m").item())/28.5, ' standar devations from the book solution.')
